In [6]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import FAISS
#from langchain_community.vectorstores import Chroma

from langchain.text_splitter import CharacterTextSplitter,RecursiveCharacterTextSplitter

load_dotenv()
api_key  = os.getenv("OPENAI_API_KEY")

# current_dir = os.path.dirname( os.getcwd() )
# print("Dir:", current_dir)

# Loading knowledge retriever
loader = TextLoader("retrieval/about_you.txt", encoding="UTF-8")
data = loader.load()
# text_splitter = CharacterTextSplitter(
#     separator="\n\n",
#     chunk_size=300,
#     length_function=len
# )

text_splitter = RecursiveCharacterTextSplitter(
chunk_size=1000, chunk_overlap=0, separators=[" ", ",", "\n"]
)
docs = text_splitter.split_documents(data)
print("Chunks: ", len(docs))
embedings_model = OpenAIEmbeddings(model = "text-embedding-3-small")

db = FAISS.from_documents(docs,embedings_model)
retriever = db.as_retriever(search_kwargs={"k": 1})

from langchain.agents.agent_toolkits import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "about_you_information",
    "Searches and returns information about return and delivery questions for AboutYou.",
)
tools = [retriever_tool]


llm = ChatOpenAI(model="gpt-3.5-turbo",temperature=0,streaming=True)

# Define prompt
from langchain.prompts import MessagesPlaceholder
from langchain.schema.messages import SystemMessage

system_message = SystemMessage(
    content=(
        """You are a voice call customer support bot for a bulgarian clothing store called AboutYou .
Don't answer any questions outside of the domain of AboutYou custommer support.
Always execute about_you_information tool.
A user will call you and ask you questions about his delivery or return of a product.
If you don't have the answer of the question in your knowledge say that you don't know, don't try to make up information.
You give help to questions regarding the return and delivery of AboutYou items.
Answer the questions briefly and summarize your information.
Say maximum 3 sentances per answer.
Аnswer the questions in bulgarian.
"""
    )
)


from langchain.agents.openai_functions_agent.base import OpenAIFunctionsAgent
memory_key = "chat_history"
prompt = OpenAIFunctionsAgent.create_prompt(
    system_message=system_message,
    extra_prompt_messages=[MessagesPlaceholder(variable_name=memory_key)],
)

# Define final Agent 
from langchain.agents import create_openai_functions_agent

#agent = OpenAIFunctionsAgent(llm=llm, tools=tools, prompt=prompt)
agent = create_openai_functions_agent(llm, tools, prompt)

from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    return_intermediate_steps=False,
    #verbose=True
)


from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
message_history = ChatMessageHistory()
message_history.clear()

agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key=memory_key,
)

Chunks:  5


In [12]:
agent_with_chat_history.get_session_history("<foo>")

ChatMessageHistory(messages=[])

In [13]:
agent_with_chat_history.invoke({'input': 'как мога да си върна продукта'}, {'configurable': {'session_id': 'asd'}})['output']

'За да върнеш продукта си, постави го в оригиналната опаковка, приложи документа за връщане и залепи етикета за връщане. След това можеш да го изпратиш до най-близкия офис на Еконт или да поръчаш куриер.\nАко си загубил документите за връщане, можеш да ги свалиш от клиентския си профил в ABOUT YOU и да ги използваш за връщане на продукта.'

### Streaming

In [3]:
from langserve import RemoteRunnable
from langchain_community.chat_message_histories import ChatMessageHistory

remote_agent = RemoteRunnable(
    "https://bot.happytree-937aa4bb.westus2.azurecontainerapps.io"
)
message_history = ChatMessageHistory()
message_history.clear()

In [14]:
from  langchain_core.messages.ai import AIMessageChunk
async def get_tokens(input:str):
    path_status = {}
    async for chunk in agent_with_chat_history.astream_log(
        {"input": input, "chat_history": message_history.messages},
        config={"configurable": {"session_id": "<foo>"}},
        include_names=["ChatOpenAI"],
    ):
        for op in chunk.ops:
            if op["op"] == "add":
                if op["path"] not in path_status:
                    path_status[op["path"]] = op["value"]
                else:
                    if not isinstance( op["value"], dict):
                        path_status[op["path"]] += op["value"]
        if isinstance(path_status.get(op["path"]), AIMessageChunk):
            yield path_status.get(op["path"]).content

In [20]:
message_history = ChatMessageHistory()
message_history.clear()

In [21]:
import re
pattern = re.compile(r'[.!?]')
found = 0
async for token in get_tokens(input="как мога да върна продукт"):
    sentence_ends = [match.start() for match in re.finditer(pattern, token)]
    if len(sentence_ends) > found:
        text = token[:sentence_ends[0] + 1] if found == 0 else token[sentence_ends[-2]+1 :]
        print("Speaking :", text)
        found +=1


Speaking : Връщането на продукт е лесно:
1.
Speaking :  Постави артикула в неговата оригинална опаковка.

Speaking : 
2.
Speaking :  Приложи документа за връщане и залепи етикета за връщане на пакета.

Speaking : 
3.
Speaking :  Занеси пакета до най-близкия офис на Еконт или поръчай куриер.
